# ~ TilStat scraper ~

Utility to scrape the scores to enter the various courses over the years. Like any scraping utility, this code sucks.

In [57]:
import requests
from bs4 import BeautifulSoup as bs
from multiprocessing import Pool
import pandas as pd
import numpy as np

In [58]:
from datetime import datetime

In [59]:
base = 'http://didattica.polito.it'

In [60]:
indexes = []

for year in [2018,2019,2020,2021,2022,2023,2024,2025]:
    indexes += [
        {'uri':f'{base}/pls/portal30/preimma.pkg_grad.arc?p_a_acc={year}',
         'is_contingentato': False, 'course': 'Architettura', 'year':year, 'til_kind': 'TIL-A'},
        # {'uri':f'{base}/pls/portal30/preimma.pkg_grad.arc_conting?p_a_acc={year}',
        #  'conting': True, 'course': 'Architettura', 'year':year},

        {'uri':f'{base}/pls/portal30/preimma.pkg_grad.des?p_a_acc={year}',
         'is_contingentato': False, 'course': 'Design', 'year':year, 'til_kind': 'TIL-D'},
        # {'uri':f'{base}/pls/portal30/preimma.pkg_grad.des_cont?p_a_acc={year}',
        #  'conting': True, 'course': 'Design', 'year':year},

        {'uri':f'{base}/pls/portal30/preimma.pkg_grad.pian?p_a_acc={year}',
         'is_contingentato': False, 'course': 'Pianificazione', 'year':year, 'til_kind': 'TIL-P'},
        # {'uri':f'{base}/pls/portal30/preimma.pkg_grad.pian_cont?p_a_acc={year}',
        #  'conting': True, 'course': 'Pianificazione', 'year':year}, 
    ]
    

for year in [2018,2019,2020,2021]:
    indexes += [
        {'uri':f'{base}/pls/portal30/preimma.pkg_grad.ing?p_a_acc={year}',
         'is_contingentato': False, 'course': 'Ingegneria', 'year':year, 'til_kind': 'TIL-I'},
        # {'uri':f'{base}/pls/portal30/preimma.pkg_grad.ing_cont?p_a_acc={year}',
        #  'conting': True, 'course': 'Ingegneria', 'year':year},
    ]

In [61]:
fogli_graduatorie = []
for index in indexes:
    html = requests.get(index['uri']).text
    soup = bs(html, 'html.parser')
    ranklinks = [link for link in soup.find_all("a")
                     if "href" in link.attrs and 'preimma' in link['href'] and link.text.startswith('Graduatoria del ')]
    
    for i, link in enumerate(ranklinks):
        
        # paranoid check
        # Es: Ingegneria has .ing string in each rank
        assert f".{index['course'].lower()[:3]}" in link['href'], "Wrong link assoc..."
        
        date = [int(x) for x in link.text.replace("Graduatoria del ","").split('/')]
        date = datetime(date[2],date[1],date[0])
        
        # paranoid check
        first_sane_rank = datetime(int(index['year'])-1, 7, 1) # dal primo luglio del primo anno di inizio coorte
        last_sane_rank = datetime(int(index['year']), 3, 15) # a metà marzo del secondo anno di inizio coorte
        assert first_sane_rank < date < last_sane_rank, "Wrong year assoc..."
        
        fogli_graduatorie.append({
            'grad_url': base+link['href'],
            'til_kind': index['til_kind'],
            'course': index['course'],
            'is_contingentato': index['is_contingentato'],
            'grad_n': i,
            'date': date,
            'year': index['year']
        })

In [62]:
x=0
def scrapeGraduatoria(rank):
    global x
    x+=1
    print(x)
    
    html = requests.get(rank['grad_url']).text
    soup = bs(html, 'html.parser')
    
    headers = soup.select("table th")
    rows = soup.select("table tr")
    cells = soup.select("table td")

    # paranoid check
    assert len(headers)*(len(rows)-1) == len(cells), "Wrong table format"

    headervalues = [str(h.text.lower().strip()) for h in headers]
    values = [str(c.text.strip()) for c in cells]
    shapedvalues = np.asarray(values).reshape((len(rows)-1),len(headers)).astype(str)
    df = pd.DataFrame(shapedvalues, columns=headervalues)

    df = df[df['totale'] != "Certificazione Internazionale"]
    df["totale"] = df["totale"].str.replace(',','.').astype(float)
    
    df['grad_url'] = rank['grad_url']
    df['course'] = rank['course']
    df['year'] = rank['year']
    df['grad_n'] = rank['grad_n'] # ennesima graduatoria nell'anno
    df['date'] = rank['date']
    df['is_contingentato'] = rank['is_contingentato']

    # riordino df
    order = ['grad_url', 'course', 'year', 'grad_n', 'date', 'is_contingentato']
    df = df[order + [c for c in df.columns if c not in order]]

    return df

In [63]:
len(fogli_graduatorie)

211

In [64]:
dfs = list(map(scrapeGraduatoria, fogli_graduatorie))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [65]:
result = pd.concat(dfs, ignore_index=True)

In [70]:
result["af user"] = result["af user"].fillna(result["user"])
result.drop(columns=["user"], inplace=True)

In [71]:
result.to_csv(f"./intermediate_results/v1_{str(datetime.now()).replace(':','-')}.csv", sep=";")